In [2]:
#import packages
import cv2
import time
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib

from random import randint
from pathlib import Path
import json

from collections import defaultdict

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torchvision.models import resnet34
#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
from custom_datasets import YogaPoseDataset

import model_utils
import plot_utils

from imutils.video import VideoStream
from scipy.spatial.distance import euclidean
from imutils import perspective
from imutils import contours
import imutils
from scipy.spatial import distance as dist
from imutils.video import FPS
import math
import os
from tensorflow.keras.models import model_from_json
from tensorflow.keras.preprocessing import image

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier

import pyautogui
from time import time
from math import hypot
import mediapipe as mp

import pyaudio
import wave

import librosa
import librosa.display
import tensorflow as tf
import tensorflow.keras
from matplotlib.pyplot import specgram
import scipy.io.wavfile
import sys
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
import glob 

pyautogui.PAUSE = 0

In [2]:
def audioRecorder(): 
    CHUNK = 1024 
    FORMAT = pyaudio.paInt16 #paInt8
    CHANNELS = 2 
    RATE = 44100 #sample rate
    RECORD_SECONDS = 4
    WAVE_OUTPUT_FILENAME = "output11.wav"

    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK) #buffer

    print("* recording")

    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data) # 2 bytes(16 bits) per channel

    print("* done recording")

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

In [3]:
feelings = ['Calm', 'Happy', 'Sad', 'Angry', 'Fearful']
def audioEmotionDetector(): 
    config = tensorflow.ConfigProto(
    device_count={'GPU': 1},
    intra_op_parallelism_threads=1,
    allow_soft_placement=True
    )

    config.gpu_options.allow_growth = True
    config.gpu_options.per_process_gpu_memory_fraction = 0.6

    session = tensorflow.Session(config=config)
    tf.keras.backend.set_session(session)

    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("saved_models/Emotion_Voice_Detection_Model.h5")
    print("Loaded model from disk")
    data, sampling_rate = librosa.load('output10.wav')
    X, sample_rate = librosa.load('output10.wav', res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
    sample_rate = np.array(sample_rate)
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13),axis=0)
    featurelive = mfccs
    livedf2 = featurelive
    livedf2= pd.DataFrame(data=livedf2)
    livedf2 = livedf2.stack().to_frame().T
    twodim= np.expand_dims(livedf2, axis=2)
    livepreds = loaded_model.predict(twodim, 
                             batch_size=32, 
                             verbose=1)
    livepreds1=livepreds.argmax(axis=1)
    liveabc = livepreds1.astype(int).flatten()
    livefeeling = int(liveabc) - 2
    feeling = feelings[livefeeling]
    return feeling


In [4]:
#import model file and define pairs of pose
protoFile = "pose/coco/pose_deploy_linevec.prototxt"
weightsFile = "pose/coco/pose_iter_440000.caffemodel"
nPoints = 18
# COCO Output Format
keypointsMapping = ['Nose', 'Neck', 'R-Sho', 'R-Elb', 'R-Wr', 'L-Sho', 
                    'L-Elb', 'L-Wr', 'R-Hip', 'R-Knee', 'R-Ank', 'L-Hip', 
                    'L-Knee', 'L-Ank', 'R-Eye', 'L-Eye', 'R-Ear', 'L-Ear']

POSE_PAIRS = [[1,2], [1,5], [2,3], [3,4], [5,6], [6,7],
              [1,8], [8,9], [9,10], [1,11], [11,12], [12,13],
              [1,0], [0,14], [14,16], [0,15], [15,17],
              [2,17], [5,16] ]

# index of pafs correspoding to the POSE_PAIRS
# e.g for POSE_PAIR(1,2), the PAFs are located at indices (31,32) of output, Similarly, (1,5) -> (39,40) and so on.
mapIdx = [[31,32], [39,40], [33,34], [35,36], [41,42], [43,44], 
          [19,20], [21,22], [23,24], [25,26], [27,28], [29,30], 
          [47,48], [49,50], [53,54], [51,52], [55,56], 
          [37,38], [45,46]]

colors = [ [0,100,255], [0,100,255], [0,255,255], [0,100,255], [0,255,255], [0,100,255],
         [0,255,0], [255,200,100], [255,0,255], [0,255,0], [255,200,100], [255,0,255],
         [0,0,255], [255,0,0], [200,200,0], [255,0,0], [200,200,0], [0,0,0]]

In [5]:
# Find the Keypoints using Non Maximum Suppression on the Confidence Map
def getKeypoints(probMap, threshold=0.1):
    
    mapSmooth = cv2.GaussianBlur(probMap,(3,3),0,0)

    mapMask = np.uint8(mapSmooth>threshold)
    keypoints = []
    
    #find the blobs
    contours, _ = cv2.findContours(mapMask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    #contours,hierachy=cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    #for each blob find the maxima
    for cnt in contours:
        blobMask = np.zeros(mapMask.shape)
        blobMask = cv2.fillConvexPoly(blobMask, cnt, 1)
        maskedProbMap = mapSmooth * blobMask
        _, maxVal, _, maxLoc = cv2.minMaxLoc(maskedProbMap)
        keypoints.append(maxLoc + (probMap[maxLoc[1], maxLoc[0]],))

    return keypoints

In [6]:
# Find valid connections between the different joints of a all persons present
def getValidPairs(output):
    valid_pairs = []
    invalid_pairs = []
    n_interp_samples = 10
    paf_score_th = 0.1
    conf_th = 0.7
    # loop for every POSE_PAIR
    for k in range(len(mapIdx)):
        # A->B constitute a limb
        pafA = output[0, mapIdx[k][0], :, :]
        pafB = output[0, mapIdx[k][1], :, :]
        pafA = cv2.resize(pafA, (frameWidth, frameHeight))
        pafB = cv2.resize(pafB, (frameWidth, frameHeight))

        # Find the keypoints for the first and second limb
        candA = detected_keypoints[POSE_PAIRS[k][0]]
        candB = detected_keypoints[POSE_PAIRS[k][1]]
        nA = len(candA)
        nB = len(candB)

        # If keypoints for the joint-pair is detected
        # check every joint in candA with every joint in candB 
        # Calculate the distance vector between the two joints
        # Find the PAF values at a set of interpolated points between the joints
        # Use the above formula to compute a score to mark the connection valid
        
        if( nA != 0 and nB != 0):
            valid_pair = np.zeros((0,3))
            for i in range(nA):
                max_j=-1
                maxScore = -1
                found = 0
                for j in range(nB):
                    # Find d_ij
                    d_ij = np.subtract(candB[j][:2], candA[i][:2])
                    norm = np.linalg.norm(d_ij)
                    if norm:
                        d_ij = d_ij / norm
                    else:
                        continue
                    # Find p(u)
                    interp_coord = list(zip(np.linspace(candA[i][0], candB[j][0], num=n_interp_samples),
                                            np.linspace(candA[i][1], candB[j][1], num=n_interp_samples)))
                    # Find L(p(u))
                    paf_interp = []
                    for k in range(len(interp_coord)):
                        paf_interp.append([pafA[int(round(interp_coord[k][1])), int(round(interp_coord[k][0]))],
                                           pafB[int(round(interp_coord[k][1])), int(round(interp_coord[k][0]))] ]) 
                    # Find E
                    paf_scores = np.dot(paf_interp, d_ij)
                    avg_paf_score = sum(paf_scores)/len(paf_scores)
                    
                    # Check if the connection is valid
                    # If the fraction of interpolated vectors aligned with PAF is higher then threshold -> Valid Pair  
                    if ( len(np.where(paf_scores > paf_score_th)[0]) / n_interp_samples ) > conf_th :
                        if avg_paf_score > maxScore:
                            max_j = j
                            maxScore = avg_paf_score
                            found = 1
                # Append the connection to the list
                if found:            
                    valid_pair = np.append(valid_pair, [[candA[i][3], candB[max_j][3], maxScore]], axis=0)

            # Append the detected connections to the global list
            valid_pairs.append(valid_pair)
        else: # If no keypoints are detected
            print("No Connection : k = {}".format(k))
            invalid_pairs.append(k)
            valid_pairs.append([])
    print(valid_pairs)
    return valid_pairs, invalid_pairs

In [7]:
# This function creates a list of keypoints belonging to each person
# For each detected valid pair, it assigns the joint(s) to a person
# It finds the person and index at which the joint should be added. This can be done since we have an id for each joint
def getPersonwiseKeypoints(valid_pairs, invalid_pairs):
    # the last number in each row is the overall score 
    personwiseKeypoints = -1 * np.ones((0, 19))

    for k in range(len(mapIdx)):
        if k not in invalid_pairs:
            partAs = valid_pairs[k][:,0]
            partBs = valid_pairs[k][:,1]
            indexA, indexB = np.array(POSE_PAIRS[k])

            for i in range(len(valid_pairs[k])): 
                found = 0
                person_idx = -1
                for j in range(len(personwiseKeypoints)):
                    if personwiseKeypoints[j][indexA] == partAs[i]:
                        person_idx = j
                        found = 1
                        break

                if found:
                    personwiseKeypoints[person_idx][indexB] = partBs[i]
                    personwiseKeypoints[person_idx][-1] += keypoints_list[partBs[i].astype(int), 2] + valid_pairs[k][i][2]

                # if find no partA in the subset, create a new subset
                elif not found and k < 17:
                    row = -1 * np.ones(19)
                    row[indexA] = partAs[i]
                    row[indexB] = partBs[i]
                    # add the keypoint_scores for the two keypoints and the paf_score 
                    row[-1] = sum(keypoints_list[valid_pairs[k][i,:2].astype(int), 2]) + valid_pairs[k][i][2]
                    personwiseKeypoints = np.vstack([personwiseKeypoints, row])
    return personwiseKeypoints

In [8]:
#Directory of dataset
JPEG_DIR = Path('dataset/')
#Reading JSON file
pose_list = json.load(Path('pose-list-with-meta.json').open())
#Encode pose
pose_id_to_name = {0: 'Bharadvajasana I', 1: 'Padangusthasana', 2: 'Paripurna Navasana', 3: 'Baddha Konasana', 4: 'Dhanurasana', 5: 'Setu Bandha Sarvangasana', 6: 'Ustrasana', 7: 'Marjaryasana', 8: 'Chakravakasana', 9: 'Ashtanga Namaskara', 10: 'Utkatasana', 11: 'Balasana', 12: 'Bhujangasana', 13: 'Savasana', 14: 'Gomukhasana', 15: 'Bitilasana', 16: 'Bakasana', 17: 'Makara Adho Mukha Svanasana', 18: 'Ardha Pincha Mayurasana', 19: 'Adho Mukha Svanasana', 20: 'Garudasana', 21: 'Sukhasana', 22: 'Astavakrasana', 23: 'Utthita Hasta Padangustasana', 24: 'Uttana Shishosana', 25: 'Utthita Parsvakonasana', 26: 'Utthita Trikonasana', 27: 'Pincha Mayurasana', 28: 'Agnistambhasana', 29: 'Tittibhasana', 30: 'Matsyasana', 31: 'Chaturanga Dandasana', 32: 'Malasana', 33: 'Parighasana', 34: 'Ardha Bhekasana', 35: 'Ardha Matsyendrasana', 36: 'Supta Matsyendrasana', 37: 'Ardha Chandrasana', 38: 'Adho Mukha Vriksasana', 39: 'Ananda Balasana', 40: 'Janu Sirsasana', 41: 'Virasana', 42: 'Krounchasana', 43: 'Utthita Ashwa Sanchalanasana', 44: 'Parsvottanasana', 45: 'Viparita Karani', 46: 'Salabhasana', 47: 'Natarajasana', 48: 'Padmasana', 49: 'Anjaneyasana', 50: 'Marichyasana III', 51: 'Hanumanasana', 52: 'Tadasana', 53: 'Pasasana', 54: 'Eka Pada Rajakapotasana', 55: 'Eka Pada Rajakapotasana II', 56: 'Mayurasana', 57: 'Kapotasana', 58: 'Phalakasana', 59: 'Halasana', 60: 'Eka Pada Koundinyanasana I', 61: 'Eka Pada Koundinyanasana II', 62: 'Marichyasana I', 63: 'Supta Baddha Konasana', 64: 'Supta Padangusthasana', 65: 'Supta Virasana', 66: 'Parivrtta Janu Sirsasana', 67: 'Parivrtta Parsvakonasana', 68: 'Parivrtta Trikonasana', 69: 'Tolasana', 70: 'Paschimottanasana', 72: 'Parsva Bakasana', 73: 'Vasisthasana', 74: 'Anantasana', 75: 'Salamba Bhujangasana', 76: 'Dandasana', 77: 'Uttanasana', 78: 'Ardha Uttanasana', 79: 'Urdhva Prasarita Eka Padasana', 80: 'Salamba Sirsasana', 81: 'Salamba Sarvangasana', 82: 'Vriksasana', 83: 'Urdhva Dhanurasana', 84: 'Dwi Pada Viparita Dandasana', 85: 'Purvottanasana', 86: 'Urdhva Hastasana', 87: 'Urdhva Mukha Svanasana', 88: 'Virabhadrasana I', 89: 'Virabhadrasana II', 90: 'Virabhadrasana III', 91: 'Upavistha Konasana', 92: 'Prasarita Padottanasana', 93: 'Camatkarasana', 94: 'Yoganidrasana', 95: 'Vrischikasana', 96: 'Vajrasana', 97: 'Tulasana', 98: 'Simhasana', 99: 'Makarasana', 100: 'Lolasana', 101: 'Kurmasana', 102: 'Garbha Pindasana', 103: 'Durvasasana', 71: 'Bhujapidasana', 104: 'Bhekasana', 105: 'Bhairavasana', 106: 'Ganda Bherundasana'}
#Decode pose
pose_name_to_id = {'bharadvajasana i': 0, 'padangusthasana': 1, 'paripurna navasana': 2, 'baddha konasana': 3, 'dhanurasana': 4, 'setu bandha sarvangasana': 5, 'ustrasana': 6, 'marjaryasana': 7, 'chakravakasana': 8, 'ashtanga namaskara': 9, 'utkatasana': 10, 'balasana': 11, 'bhujangasana': 12, 'savasana': 13, 'gomukhasana': 14, 'bitilasana': 15, 'bakasana': 16, 'makara adho mukha svanasana': 17, 'ardha pincha mayurasana': 18, 'adho mukha svanasana': 19, 'garudasana': 20, 'sukhasana': 21, 'astavakrasana': 22, 'utthita hasta padangustasana': 23, 'uttana shishosana': 24, 'utthita parsvakonasana': 25, 'utthita trikonasana': 26, 'pincha mayurasana': 27, 'agnistambhasana': 28, 'tittibhasana': 29, 'matsyasana': 30, 'chaturanga dandasana': 31, 'malasana': 32, 'parighasana': 33, 'ardha bhekasana': 34, 'ardha matsyendrasana': 35, 'supta matsyendrasana': 36, 'ardha chandrasana': 37, 'adho mukha vriksasana': 38, 'ananda balasana': 39, 'janu sirsasana': 40, 'virasana': 41, 'krounchasana': 42, 'utthita ashwa sanchalanasana': 43, 'parsvottanasana': 44, 'viparita karani': 45, 'salabhasana': 46, 'natarajasana': 47, 'padmasana': 48, 'anjaneyasana': 49, 'marichyasana iii': 50, 'hanumanasana': 51, 'tadasana': 52, 'pasasana': 53, 'eka pada rajakapotasana': 54, 'eka pada rajakapotasana ii': 55, 'mayurasana': 56, 'kapotasana': 57, 'phalakasana': 58, 'halasana': 59, 'eka pada koundinyanasana i': 60, 'eka pada koundinyanasana ii': 61, 'marichyasana i': 62, 'supta baddha konasana': 63, 'supta padangusthasana': 64, 'supta virasana': 65, 'parivrtta janu sirsasana': 66, 'parivrtta parsvakonasana': 67, 'parivrtta trikonasana': 68, 'tolasana': 69, 'paschimottanasana': 70, 'parsva bakasana': 72, 'vasisthasana': 73, 'anantasana': 74, 'salamba bhujangasana': 75, 'dandasana': 76, 'uttanasana': 77, 'ardha uttanasana': 78, 'urdhva prasarita eka padasana': 79, 'salamba sirsasana': 80, 'salamba sarvangasana': 81, 'vriksasana': 82, 'urdhva dhanurasana': 83, 'dwi pada viparita dandasana': 84, 'purvottanasana': 85, 'urdhva hastasana': 86, 'urdhva mukha svanasana': 87, 'virabhadrasana i': 88, 'virabhadrasana ii': 89, 'virabhadrasana iii': 90, 'upavistha konasana': 91, 'prasarita padottanasana': 92, 'camatkarasana': 93, 'yoganidrasana': 94, 'vrischikasana': 95, 'vajrasana': 96, 'tulasana': 97, 'simhasana': 98, 'makarasana': 99, 'lolasana': 100, 'kurmasana': 101, 'garbha pindasana': 102, 'durvasasana': 103, 'bhujapidasana': 71, 'bhekasana': 104, 'bhairavasana': 105, 'ganda bherundasana': 106}
#Read classifier csv file
classifier_csv = pd.read_csv('classifier_train_data.csv')

In [9]:
#Define resize method
from PIL import ImageOps
from PIL import Image

class Resize(object):
    def __init__(self, size=224):
        self.size = size
        
    def __call__(self, im):  
        if(im.height > im.width):
            
            w = int(self.size*im.width/im.height)
            h = self.size
            pad_val = int((224-w)/2)
            pad = (224-w-pad_val,0,pad_val,0)
        else:
            h = int(self.size*im.height/im.width)
            w = self.size
            pad_val = int((224-h)/2)
            pad = (0,224-h-pad_val,0,pad_val)
        return ImageOps.expand(im.resize((w,h),resample=Image.BILINEAR), pad)

In [10]:
#Transform
bs = 32
sz = 224
n_epochs = 1
lr = 0.001

trn_tfms = transforms.Compose([
    Resize(),
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor()
])

val_tfms = transforms.Compose([
    Resize(),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor()
])

In [11]:
p = transforms.ToPILImage()
#p(trn_tfms(im))

In [12]:
#Define data loader
dl = torch.utils.data.DataLoader(YogaPoseDataset('classifier_train_data.csv', trn_tfms, 
                                                           pose_id_to_name, pose_name_to_id),
                                           bs, 
                                           shuffle=True)

In [13]:
#Model read
model = resnet34(pretrained=True)
model.fc = nn.Linear(512, 107)   
model_utils.freeze_all_layers(model)
#Making require_grade is true
for param in model.parameters():
    param.requires_grad = True   
#Define Model
model = resnet34(pretrained=True)
model.fc = nn.Linear(512, 107)   
#model = model.to(device)
model.load_state_dict(torch.load('./models/yoga-asana-classifier.ckpt', map_location='cpu'))


<All keys matched successfully>

In [14]:
#define variable
val_tfms = transforms.Compose([
    Resize(),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor()
])

In [15]:
#Method to evaluate image on model
def evaluate_single_image_on_model(model, val_tfms, test_img, pose_id_to_name):
    model.eval()
    with torch.no_grad():
        res = model(val_tfms(test_img)[None])
        pred_probs = torch.nn.functional.softmax(res, dim=1)[0]
        top_two = pred_probs.argsort()[-2:].numpy()
        
        res = []
        
        if(pred_probs[top_two[-1]] > 0.8):
            res.append(pose_id_to_name[top_two[-1]])
        else:
            res.append(pose_id_to_name[top_two[-1]])
            res.append(pose_id_to_name[top_two[-2]])
        
        return res
        '''
        res = []
        if(pred_probs[top_two[-1]]>0.8):
            res.append({'conf':round(pred_probs[top_two[-1]].item(), 2), 'pose_name': pose_id_to_name[top_two[-1]]})
        else:
            res.append({'conf':round(pred_probs[top_two[-1]].item(), 2), 'pose_name': pose_id_to_name[top_two[-1]]})
            res.append({'conf':round(pred_probs[top_two[-2]].item(), 2), 'pose_name': pose_id_to_name[top_two[-2]]})
        return res
        '''

In [16]:
def capture():  
    # initialize the camera
    # If you have multiple camera connected with 
    # current device, assign a value in cam_port 
    # variable according to that
    cam = cv2.VideoCapture(0)

    # reading the input using the camera
    result, image = cam.read()

    # If image will detected without any error, 
    # show result
    if result:

        # showing result, it take frame name and image 
        # output
        cv2.imshow("Yoga", image)

        # saving image in local storage
        cv2.imwrite("YogaPose.png", image)

        # If keyboard interrupt occurs, destroy image 
        # window
        cv2.waitKey(0)
        cv2.destroyWindow("Yoga")

    # If captured image is corrupted, moving to else part
    else:
        print("No image detected. Please! try again")

In [17]:
# open image
def yogaPredictor():
    test_img = Image.open('test5.png') # 'YogaPose.png'
    # call evaluate method to show name of pose
    yogaPose = evaluate_single_image_on_model(model, val_tfms, test_img, pose_id_to_name)
    if yogaPose[0] == 'Bhujapidasana':
        return "Green"
    else:
        return "Red"

In [18]:
# Importing db of food items across all canteens registered on the platform
df1=pd.read_csv('food db/food.csv')
df1.columns = ['food_id','title','canteen_id','price', 'num_orders', 'category', 'avg_rating', 'num_rating', 'tags']
# mean of average ratings of all items
C= df1['avg_rating'].mean()

# the minimum number of votes required to appear in recommendation list, i.e, 60th percentile among 'num_rating'
m= df1['num_rating'].quantile(0.6)

# items that qualify the criteria of minimum num of votes
q_items = df1.copy().loc[df1['num_rating'] >= m]

# Calculation of weighted rating based on the IMDB formula
def weighted_rating(x, m=m, C=C):
    v = x['num_rating']
    R = x['avg_rating']
    return (v/(v+m) * R) + (m/(m+v) * C)

# Applying weighted_rating to qualified items
q_items['score'] = q_items.apply(weighted_rating, axis=1)

# Shortlisting the top rated items and popular items
top_rated_items = q_items.sort_values('score', ascending=False)
pop_items= df1.sort_values('num_orders', ascending=False)

# Display results of demographic filtering
top_rated_items[['title', 'num_rating', 'avg_rating', 'score']].head()
pop_items[['title', 'num_orders']].head()
# Creating soup string for each item
def create_soup(x):            
    tags = x['tags'].lower().split(', ')
    tags.extend(x['title'].lower().split())
    tags.extend(x['category'].lower().split())
    return " ".join(sorted(set(tags), key=tags.index))

df1['soup'] = df1.apply(create_soup, axis=1)
df1.head(3)

# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(stop_words='english')

# df1['soup']
count_matrix = count.fit_transform(df1['soup'])

# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(count_matrix, count_matrix)

indices_from_title = pd.Series(df1.index, index=df1['title'])
indices_from_food_id = pd.Series(df1.index, index=df1['food_id'])

# Function that takes in food title or food id as input and outputs most similar dishes 
def get_recommendations(title="", cosine_sim=cosine_sim, idx=-1):
    # Get the index of the item that matches the title
    if idx == -1 and title != "":
        idx = indices_from_title[title]

    # Get the pairwsie similarity scores of all dishes with that dish
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the dishes based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the scores of the 10 most similar dishes
    sim_scores = sim_scores[1:3]

    # Get the food indices
    food_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar dishes
    return food_indices

df1.loc[get_recommendations(title="Paneer Tikka")]
# fetch few past orders of a user, based on which personalized recommendations are to be made
def get_latest_user_orders(user_id, orders, num_orders=3):
    counter = num_orders
    order_indices = []
    
    for index, row in orders[['user_id']].iterrows():
        if row.user_id == user_id:
            counter = counter -1
            order_indices.append(index)
        if counter == 0:
            break
            
    return order_indices

# utility function that returns a DataFrame given the food_indices to be recommended
def get_recomms_df(food_indices, df1, columns, comment):
    row = 0
    df = pd.DataFrame(columns=columns)
    
    for i in food_indices:
        df.loc[row] = df1[['title', 'canteen_id', 'price']].loc[i]
        df.loc[row].comment = comment
        row = row+1
    return df

# return food_indices for accomplishing personalized recommendation using Count Vectorizer
def personalised_recomms(orders, df1, user_id, columns, comment="based on your past orders"):
    order_indices = get_latest_user_orders(user_id, orders)
    food_ids = []
    food_indices = []
    recomm_indices = []
    
    for i in order_indices:
        food_ids.append(orders.loc[i].food_id)
    for i in food_ids:
        food_indices.append(indices_from_food_id[i])
    for i in food_indices:
        recomm_indices.extend(get_recommendations(idx=i))
        
    return get_recomms_df(set(recomm_indices), df1, columns, comment)

# Simply fetch new items added by vendor or today's special at home canteen
def get_new_and_specials_recomms(new_and_specials, users, df1, canteen_id, columns, comment="new/today's special item  in your home canteen"):
    food_indices = []
    
    for index, row in new_and_specials[['canteen_id']].iterrows():
        if row.canteen_id == canteen_id:
            food_indices.append(indices_from_food_id[new_and_specials.loc[index].food_id])
            
    return get_recomms_df(set(food_indices), df1, columns, comment)

# utility function to get the home canteen given a user id
def get_user_home_canteen(users, user_id):
    for index, row in users[['user_id']].iterrows():
        if row.user_id == user_id:
            return users.loc[index].home_canteen
    return -1

# fetch items from previously calculated top_rated_items list
def get_top_rated_items(top_rated_items, df1, columns, comment="top rated items across canteens"):
    food_indices = []
    
    for index, row in top_rated_items.iterrows():
        food_indices.append(indices_from_food_id[top_rated_items.loc[index].food_id])
        
    return get_recomms_df(food_indices, df1, columns, comment)

# fetch items from previously calculated pop_items list
def get_popular_items(pop_items, df1, columns, comment="most popular items across canteens"):
    food_indices = []
    
    for index, row in pop_items.iterrows():
        food_indices.append(indices_from_food_id[pop_items.loc[index].food_id])
        
    return get_recomms_df(food_indices, df1, columns, comment)
    
orders = pd.read_csv('food db/orders.csv')
new_and_specials = pd.read_csv('food db/new_and_specials.csv')
users = pd.read_csv('food db/users.csv')

columns = ['title', 'price', 'comment']
current_user = 2
current_canteen = get_user_home_canteen(users, current_user)

# get_popular_items(pop_items, df1, columns)

# get_top_rated_items(top_rated_items, df1, columns)

# get_new_and_specials_recomms(new_and_specials, users, df1, current_canteen, columns)

# personalised_recomms(orders, df1, current_user, columns)

In [19]:
data=pd.read_csv('music.zip',compression='zip')
data.drop_duplicates(inplace=True,subset=['name'])
name=data['name']
col_features = ['danceability', 'energy', 'valence', 'loudness']
X = MinMaxScaler().fit_transform(data[col_features])
kmeans = KMeans(init="k-means++",
                n_clusters=2,
                random_state=15).fit(X)
data['kmeans'] = kmeans.labels_
data['song_name']=name
cluster=data.groupby(by=data['kmeans'])
y=data.pop('kmeans')
x=data.drop(columns=['name','artists','id','release_date','song_name'])
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25)

model=LGBMClassifier().fit(x_train,y_train)
df=cluster.apply(lambda x: x.sort_values(["popularity"],ascending=False))
df.reset_index(level=0, inplace=True) 

In [20]:
def get_results(emotion_code):
    NUM_RECOMMEND=10
    happy_set=[]
    sad_set=[]
    if emotion_code==0:
        happy_set.append(df[df['kmeans']==0]['song_name'].head(NUM_RECOMMEND))
        return pd.DataFrame(happy_set).T
    else:
        sad_set.append(df[df['kmeans']==1]['song_name'].head(NUM_RECOMMEND))
        return pd.DataFrame(sad_set).T

In [21]:
def videoEmotionDetector():
    model = model_from_json(open("fer.json", "r").read())
    model.load_weights('fer.h5')
    face_haar_cascade = cv2.CascadeClassifier(r'haarcascade_frontalface_default.xml')
    cap=cv2.VideoCapture(0)

    while True:
        ret,test_img=cap.read()
        if not ret:
            continue
        gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

        faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)
        for (x,y,w,h) in faces_detected:
            cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=7)
            roi_gray=gray_img[y:y+w,x:x+h]
            roi_gray=cv2.resize(roi_gray,(48,48))
            img_pixels = image.img_to_array(roi_gray)
            img_pixels = np.expand_dims(img_pixels, axis = 0)
            img_pixels /= 255
            predictions = model.predict(img_pixels)
            max_index = np.argmax(predictions[0])

            emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
            predicted_emotion = emotions[max_index]
            cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

        resized_img = cv2.resize(test_img, (1000, 700))
        cv2.imshow('Facial emotion analysis ',resized_img)

        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break
    cap.release()
    cv2.destroyAllWindows
    return predicted_emotion

In [22]:
mp_pose = mp.solutions.pose
 
pose_image = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5, model_complexity=1)
 
pose_video = mp_pose.Pose(static_image_mode=False, model_complexity=1, min_detection_confidence=0.7,
                          min_tracking_confidence=0.7)

mp_drawing = mp.solutions.drawing_utils 


In [23]:
def put_fps(image, time1):
    time2 = time()
    if (time2 - time1) > 0:
        frames_per_second = 1.0 / (time2 - time1)
        cv2.putText(image, 'FPS: {}'.format(int(frames_per_second)), (10, 30),cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 3)
    return time2
    

In [24]:
def detect_pose(image, draw=False):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose_video.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if draw:
        mp_drawing.draw_landmarks(image=image, landmark_list=results.pose_landmarks,
                                  connections=mp_pose.POSE_CONNECTIONS,
                                  landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255),
                                                                               thickness=3, circle_radius=3),
                                  connection_drawing_spec=mp_drawing.DrawingSpec(color=(49,125,237),
                                                                               thickness=2, circle_radius=2))
    return image, results

In [25]:
def control_game(image, landmarks):
    height, width, _ = image.shape

    left_wrist = int(landmarks[mp_pose.PoseLandmark.LEFT_WRIST].y * height)
    right_wrist = int(landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].y * height)
    nose_pose = int(landmarks[mp_pose.PoseLandmark.NOSE].y * height)

    JUMP_THRESH = 130
    IDLE_THRESH = 290

    control = 'idle'
    if nose_pose < JUMP_THRESH and (left_wrist < IDLE_THRESH or right_wrist < IDLE_THRESH):
        control = 'x'
    elif right_wrist < IDLE_THRESH and left_wrist > right_wrist:
        control = 'a'
    elif left_wrist < IDLE_THRESH and left_wrist < right_wrist:
        control = 's'
    elif nose_pose < JUMP_THRESH:
        control = 'up'
    elif nose_pose > IDLE_THRESH:
        control = 'down'

    cv2.line(image, (0, JUMP_THRESH), (width - 1, JUMP_THRESH), (255, 255, 255), 2)
    cv2.line(image, (0, IDLE_THRESH), (width - 1, IDLE_THRESH), (255, 255, 255), 2)
    cv2.putText(image, 'ACTION: {}'.format(control), (400, 30),cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 3)
    return image, control

In [26]:
def press(key):
    pyautogui.keyDown(key)
    pyautogui.sleep(0.01)
    pyautogui.keyUp(key)

In [27]:
def press_key(control, pause):
    PAUSE_DURATION = 2
    if control == 's' and pause <= 0:
        press('s')
        pause = PAUSE_DURATION
    elif control == 'a' and pause <= 0:
        press('a')
        pause = PAUSE_DURATION
    elif control == 'x' and pause <= 0:
        press('x')
        pause = PAUSE_DURATION
    elif control == 'up' and pause <= 0:
        press('up')
        pause = PAUSE_DURATION
    elif control == 'down' and pause <= 0:
        press('down')
        pause = PAUSE_DURATION
    return pause

In [28]:
def playGame1():
    cap = cv2.VideoCapture(1)

    cv2.namedWindow('Play Active', cv2.WINDOW_NORMAL)

    time1 = time()
    pause = 0
    while cap.isOpened():

        ok, frame = cap.read()

        if not ok:
            continue

        frame = cv2.flip(frame, 1)
        # control = 's'

        # frame_height, frame_width, _ = frame.shape
        frame, results = detect_pose(frame, True)

        try:
            landmarks = results.pose_landmarks.landmark
            frame, control = control_game(frame, landmarks)
            pause = press_key(control, pause)
        except:
            pass

        time1 = put_fps(frame, time1)

        cv2.imshow('Play Active', frame)

        pause -= 1
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

In [29]:
def playGame2():
    cap0 = cv2.VideoCapture(0)
    
    cap1 = cv2.VideoCapture(1)

    time1 = time()
    time2 = time()
    pause = 0
    while True:

        ok0, frame0 = cap0.read()
        ok1, frame1 = cap1.read()

        frame0 = cv2.flip(frame0, 1)
        frame1 = cv2.flip(frame1, 1)
        # control = 's'

        # frame_height, frame_width, _ = frame.shape
        frame0, results0 = detect_pose(frame0, True)
        frame1, results1 = detect_pose(frame1, True)

        try:
            landmarks0 = results0.pose_landmarks.landmark
            frame0, control0 = control_game(frame0, landmarks0)
            # pause = press_key(control0, pause)
        except:
            pass
        
        try:
            landmarks1 = results1.pose_landmarks.landmark
            frame1, control1 = control_game(frame1, landmarks1)
        except:
            pass

        time1 = put_fps(frame0, time1)
        time2 = put_fps(frame1, time2)

        cv2.imshow('Play Active', frame0)
        cv2.imshow('Player 2', frame1)

        # pause -= 1
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break

    cap0.release()
    cap1.release()
    cv2.destroyAllWindows()

In [26]:
from flask import Flask, redirect,url_for, request, render_template
app=Flask(__name__,template_folder=r'C:\Users\Ishan\Desktop\AI\template', static_folder=r'C:\Users\Ishan\Desktop\AI\static')

@app.route('/',methods=['POST','GET'])
def home():
    return render_template('index.html')

@app.route('/recordAudio', methods = ['POST','GET'])
def recorder():
    audioRecorder()
    return redirect(url_for('audioEmotion'))

@app.route('/audioEmotion', methods=['POST','GET'])
def audioEmotion():
    emotionDetected = audioEmotionDetector()
    print(emotionDetected)
    return render_template('emotionDetected.html', emotion = emotionDetected)

@app.route('/videoEmotion', methods=['POST','GET'])
def videoEmotion():
    emotionDetected = videoEmotionDetector()
    print(emotionDetected)
    return render_template('emotionDetected.html', emotion = emotionDetected)

@app.route('/musicBase', methods=['POST', 'GET'])
def musicBase():
    return render_template('musicBase.html')

@app.route('/music', methods = ['POST','GET'])
def music():
    # emotion_word = emotionDetected
    emotion_word = 'happy'
    if emotion_word=='sad':
        emotion_code=0
    else:
        emotion_code=1
    if emotion_word=='sad':
        print('emotion detected is SAD')
    else:
        print('emotion detected is HAPPY')
    musicRecommend = get_results(emotion_code)
    musicRecommend.reset_index(drop=True, inplace=True)
    return render_template('music.html', tables=[musicRecommend.to_html(classes='data', header='true')])

@app.route('/yogaBase', methods = ['GET', 'POST'])
def yogaBase():
    return render_template('yogaBase.html')

@app.route('/yoga', methods = ['POST', 'GET'])
def yoga():
    # time.sleep(5)
    # capture()
    yogaPosePredicted = yogaPredictor()
    return render_template('yoga.html', yogaPose = yogaPosePredicted)
    
@app.route('/foodBase', methods = ['POST', 'GET'])   # Food Home Page
def foodBase():
    return render_template('foodBase.html')

@app.route('/foodR1', methods = ['POST', 'GET'])
def foodR1():
    foodR1 = get_popular_items(pop_items, df1, columns)
    return render_template('foodR1.html',  tables=[foodR1.to_html(classes='data', header="true")])

@app.route('/foodR2', methods = ['POST', 'GET'])
def foodR2():
    foodR2 = get_top_rated_items(top_rated_items, df1, columns)
    return render_template('foodR2.html', tables =[foodR2.to_html(classes='data', header='true')])

@app.route('/foodR3', methods = ['POST', 'GET'])
def foodR3():
    foodR3 = get_new_and_specials_recomms(new_and_specials, users, df1, current_canteen, columns)
    return render_template('/foodR3.html', tables = [foodR3.to_html(classes='data', header='true')])

@app.route('/foodR4', methods = ['POST', 'GET'])
def foodR4():
    foodR4 = personalised_recomms(orders, df1, current_user, columns)
    return render_template('/foodR4.html', tables = [foodR4.to_html(classes='data', header = 'true')])

@app.route('/gameZone', methods = ['POST','GET'])
def gameZone():
    return render_template('/gameZone.html')

@app.route('/gamePlayer1', methods = ['POST','GET'])
def gamePlayer1():
    playGame1()
    return render_template('/gameZone.html')

@app.route('/gamePlayer2', methods = ['POST', 'GET'])
def gamePlayer2():
    playGame2()
    return render_template('/gameZone.html')


app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Apr/2022 04:59:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2022 04:59:23] "GET /gameZone HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2022 04:59:23] "GET /js/jarallax.js HTTP/1.1" 404 -
127.0.0.1 - - [07/Apr/2022 04:59:23] "GET /js/responsiveslides.min.js HTTP/1.1" 404 -
127.0.0.1 - - [07/Apr/2022 04:59:23] "GET /js/move-top.js HTTP/1.1" 404 -
127.0.0.1 - - [07/Apr/2022 04:59:23] "GET /js/easing.js HTTP/1.1" 404 -
127.0.0.1 - - [07/Apr/2022 04:59:23] "GET /js/jquery.filterizr.js HTTP/1.1" 404 -
127.0.0.1 - - [07/Apr/2022 04:59:23] "GET /js/controls.js HTTP/1.1" 404 -
127.0.0.1 - - [07/Apr/2022 04:59:23] "GET /js/jquery.chocolat.js HTTP/1.1" 404 -
127.0.0.1 - - [07/Apr/2022 04:59:24] "GET /js/jarallax.js HTTP/1.1" 404 -
127.0.0.1 - - [07/Apr/2022 04:59:24] "GET /js/responsiveslides.min.js HTTP/1.1" 404 -
127.0.0.1 - - [07/Apr/2022 04:59:24] "GET /js/jquery.filterizr.js HTTP/1.1" 404 -
127.0.0.1 - - [07/Apr/2022 